In [ ]:
from Crypto.Util.number import long_to_bytes

# Signature handler (different hash of handshake hash)
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random
from cipher_suites import *

from Crypto.Cipher import AES
from cryptography import x509
from babyTLSUtil import *
from key_exchange import *
from packets import *
from os import urandom
import socketserver
import signal
import json
from prints import *

# Certificate loading: https://cryptography.io/en/latest/x509/reference/#cryptography.x509.Certificate

class Handler(socketserver.BaseRequestHandler):

    def handle(self):
        signal.alarm(0)
        main(self.request)


class ReusableTCPServer(socketserver.ForkingMixIn, socketserver.TCPServer):
    pass


def sendMessage(s, msg):
    s.send(msg.encode())


def receiveMessage(s, msg):
    sendMessage(s, msg)
    return s.recv(4096).decode().strip()


def main(s):
    sendMessage(s, "Welcome to the H3xTEL babyTLS server!\n")
    # ----- CIPHER SUITE NEGOTIATION -----
    '''
        We assume that the server does not change the cipher suite to the client
        i.e. the cipher suite used is the one that the client send
    '''
    client_hello = receiveMessage(s, "Send your client hello: ")
    client_hello, client_cipher_suite, client_KEP = extract_client_hello(client_hello)
    print_results("Client hello", client_hello)
    client_public_key = client_KEP[0]
    encrypt_and_digest = CIPHER_SUITES[client_cipher_suite]["AUTHENTICATED_CIPHER"][0]
    decrypt_and_verify = CIPHER_SUITES[client_cipher_suite]["AUTHENTICATED_CIPHER"][1]
    HASH_ALG = CIPHER_SUITES[client_cipher_suite]["HANDSHAKE_AUTHENTICATION"][0]
    HASH_LEN = CIPHER_SUITES[client_cipher_suite]["HANDSHAKE_AUTHENTICATION"][1]
    print_results("Selected cipher suite", client_cipher_suite)
    
    
    # ----- KEY EXCHANGE -----
    KEY_EXCHANGE_ALG = client_cipher_suite.split("_")[1]
    KEY_EXCHANGE_STRUCT = client_KEP[1]
    server_hello, KEP = node_hello(KEY_EXCHANGE_ALG, KEY_EXCHANGE_STRUCT)
    sendMessage(s, f"Server hello: {server_hello}\n")
    print_results("Server hello", server_hello)
    shared_secret = KEP.generate_shared_secret(client_public_key)
    print_results("Shared secret", shared_secret)
    
    # ----- HELLO VERIFICATION -----
    handshake_keys = hello_verify([client_hello, server_hello], shared_secret, HASH_ALG, HASH_LEN)
    print_results("Handshake keys", handshake_keys) # with this keys we encrypt the rest of the handshake
    
    # ----- SERVER CERTIFICATE -----
    header = "1703030343" # Handshake header (certificate) - CHANGE THIS: depends on the data follows
    cert = open("certs/h3xtel-ctf_com.crt", "r").read() # read certificate
    print_results("H3xTEL certificate", cert)
    server_certificate_encrypted = TLS_packet(encrypt_and_digest, cert.encode(), header,
                                              handshake_keys["server_handshake_key"], 
                                              handshake_keys["server_handshake_iv"])
    sendMessage(s, f"Server certificate: {server_certificate_encrypted}\n") # The client decrypts and gets the public key of the certificate
    print_results("Server certificate", server_certificate_encrypted)
    
    # --- SERVER CERTIFICATE VERIFY -----
    signature = handshake_signature([client_hello, server_hello, cert], HASH_ALG)
    print_results("Signature", signature.hex())
    header = "1703030119" # Handshake header (certificate verify) - CHANGE THIS: depends on the data follows
    server_certificate_verify = TLS_packet(encrypt_and_digest, signature, header, handshake_keys["server_handshake_key"], 
                                                       handshake_keys["server_handshake_iv"])
    sendMessage(s, f"Server certificate verify: {server_certificate_verify}\n")
    print_results("Server certificate verify", server_certificate_verify)
    
    # ----- SERVER HANDSHAKE FINISHED -----
    header = "14000030" # Handshake header (finished) - CHANGE THIS: depends on the data follows
    finished_key = HKDF_expand_label(bytes.fromhex(handshake_keys["server_secret"]), b"finished", b"", 32, HASH_ALG)
    finished_hash = HASH_ALG(client_hello.encode() + server_hello.encode() + cert.encode() + signature).digest()
    verify_data = tls_HMAC(finished_key, finished_hash, HASH_ALG)
    print_results("Verify data for handshake finished", verify_data.hex())
    finished_encrypted, auth_tag = encrypt_and_digest(verify_data, header, handshake_keys["server_handshake_key"], 
                                                      handshake_keys["server_handshake_iv"])
    finished = {
        "Record_Header": header,
        "Encrypted_Data": finished_encrypted.hex(),
        "Auth_Tag": auth_tag.hex()
    }
    sendMessage(s, f"Server handshake finished: {json.dumps(finished)}\n")
    print_results("Server handshake finished", finished)
    
    # Master key calculation (encrypts application traffic)
    handshake_hash = HASH_ALG(client_hello.encode() + server_hello.encode() + cert.encode() + signature + verify_data).digest()
    last_result = handshake_verify(handshake_hash, bytes.fromhex(handshake_keys["handshake_secret"]), HASH_ALG, HASH_LEN)
    
    client_finished = receiveMessage(s, "Send your client handshake finished: ")
    # decrypt and verify "client_finished"
    key = handshake_keys["client_handshake_key"]
    iv = handshake_keys["client_handshake_iv"]
    client_finished = json.loads(client_finished)
    client_verify_data =  decrypt_and_verify(client_finished["Encrypted_Data"], client_finished["Record_Header"], key, iv, client_finished["Auth_Tag"])
    server_finished_key = HKDF_expand_label(bytes.fromhex(handshake_keys["client_secret"]), b"finished", b"", 32, HASH_ALG)
    server_finished_hash = HASH_ALG(client_hello.encode() + server_hello.encode() + cert.encode() + signature + verify_data).digest()
    server_verify_data = tls_HMAC(server_finished_key, server_finished_hash, HASH_ALG)
    assert server_verify_data == client_verify_data
    print("-" * 116)
    print("Handshake completed")
    
if __name__ == '__main__':
    socketserver.TCPServer.allow_reuse_address = True
    server = ReusableTCPServer(("0.0.0.0", 1337), Handler)
    server.serve_forever()

--------------------------------------------------------------------------------------------------------------------
Client hello: {"Random": "25533c20128aa05874bd7ceff7f79466", "Cipher_Suites": ["TLS_ECDHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "TLS_DHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "TLS_ECDHE_RSA_PSS_SHA256_CHACHA20_POLY1305_SHA256", "TLS_DHE_RSA_PSS_SHA256_CHACHA20_POLY1305_SHA256"], "Temporary_Cipher_Suite": "TLS_DHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "Key_Share": 1894958963742781773805972325128962580624368746667224892615077922011503686410102410455448158526242239901571201798195103382632652339764183717319377657890866013294199005762437401779963575777201712784782684762289953005330815925611215143402578526774364817425315457548759718223864476592724166988945898925068012813354856686908132345960260749702517036354780360630449723122393676194122119293317394259322883403517901010311696040821782608240550031159766993401355235494807939496517063953713986044633245185184076717808235211424386416076

Server certificate: {"Record_Header": "1703030343", "Encrypted_Data": "aa16bdcfce8313957b869c812446e33ad7613058bc0fb563a5042189777c98e7713920210993afb34555a2cee906b4dde965dfade3511ddf43a553bb3820def2bfff8d8e3d43e1c72e10ba0d516923ae351acb5ea5f82faa1277c90b49883ec94fdccf0c6cc9d9d74c1183490b7eea6a127859beb10833e408a85f007a6f8e3caf6d056ea9940b25e765aae08bc622f449a8ae6bdb671652d8bd0f5928d4b39d219d4fdbbcd1cd21709c08aae736e04277104ca0efb54dadbed7ba6f5b77de76de81ac68570ec1a5a0b10a0fea631b6d65d533b5e2439a542ce6df21aff8ff7e0fb951ff56657007ecae9820cb49a3764d044a37ec02dc27a22f2722c1d06a9cccd2a2fc741c2ce14729b270ac0b6bfd7885eccd8a2a245095c66edb91366b82f7ae3717f6f6929353036c2e44909e240cf17c0d83ce175fc92d13c5bc7a4cafaeab006e8786afc95c768806bc3640259ca97999b567c559cd5408212446789143cddf110c44d77591adc81e1e052e3494f7c799c81d9e54757b8aea1e3b8dec368a2c92e27d25d39e535ded64c4c2d9ae65eb5c3a2a0b79678ffa1c09295cdc4c9a3afc8c04e51513818ef00d0339ce2a1bdc12058e11ca4f425a0d7f8568820885ee83430f5f97a125e4997a3fdcb3d